In [1]:
# This is a debug file from Eureka
import os

# Define the new working directory relative to the current working directory
# or use an absolute path
new_working_directory = os.path.join(os.getcwd(), '..', '..', '..') # Set to MLAgentBenhc
print("New working directory: ", new_working_directory)

# Change the current working directory
os.chdir(new_working_directory)
print("New Working Directory:", os.getcwd())

from MLAgentBench_v2.agents.agent import Agent
import numpy as np 
import json
import logging 
import os
import re
import subprocess
from pathlib import Path
import shutil
import time 
import types
import copy
import ast
import matplotlib.pyplot as plt

New working directory:  c:\Users\kevihuang\OneDrive - Microsoft\Desktop\projects\MLAgentBench\MLAgentBench_v2\agents\Debug\..\..\..
New Working Directory: c:\Users\kevihuang\OneDrive - Microsoft\Desktop\projects\MLAgentBench


In [2]:
# Instantiate an environment
from types import SimpleNamespace
from MLAgentBench_v2.environment import Environment

args = SimpleNamespace(
    task='home-data-for-ml-course',
    task_type='kaggle',
    log_dir='logs/manual_debug',
    work_dir='workspace',
    max_steps=50,
    max_time=18000,
    device=0,
    python='/home/user/micromamba/envs/autogpt/bin/python',
    interactive=False,
    resume=None,
    resume_step=0,
    agent_type='DebugAgent', # Just for instantiation -- doesn't actually do anything
    # llm_name='gpt-3.5-turbo-1106',
    # fast_llm_name='gpt-3.5-turbo-1106',
    # edit_script_llm_name='gpt-3.5-turbo-1106',
    llm_name='gpt-4-1106-preview',
    fast_llm_name='gpt-4-1106-preview',
    edit_script_llm_name='gpt-4-1106-preview',
    edit_script_llm_max_tokens=4000,
    agent_max_steps=50,
    actions_remove_from_prompt=[],
    actions_add_to_prompt=[],
    no_retrieval=False,
    valid_format_entires=None,
    max_steps_in_context=3,
    max_observation_steps_in_context=3,
    max_retries=4,
    langchain_agent='zero-shot-react-description'
)

env = Environment(args)

No module named 'helm'
Could not load CRFM API key crfm_api_key.txt.
[Errno 2] No such file or directory: 'claude_api_key.txt'
Could not load anthropic API key claude_api_key.txt.
Initializing environment...
args namespace(task='home-data-for-ml-course', task_type='kaggle', log_dir='logs/manual_debug', work_dir='workspace', max_steps=50, max_time=18000, device=0, python='/home/user/micromamba/envs/autogpt/bin/python', interactive=False, resume=None, resume_step=0, agent_type='DebugAgent', llm_name='gpt-4-1106-preview', fast_llm_name='gpt-4-1106-preview', edit_script_llm_name='gpt-4-1106-preview', edit_script_llm_max_tokens=4000, agent_max_steps=50, actions_remove_from_prompt=[], actions_add_to_prompt=[], no_retrieval=False, valid_format_entires=None, max_steps_in_context=3, max_observation_steps_in_context=3, max_retries=4, langchain_agent='zero-shot-react-description')
Preparing task home-data-for-ml-course , of type:  kaggle


--- RESTORING ENVIRONMENT CHECKPOINT HERE ---



In [3]:
# Instantiate the agent
class DebugAgent(Agent):
    def __init__(self, env):
        super().__init__(env)
        self.env = env # allows you to interact with the environment using debug_agent.env

debug_agent = DebugAgent(env)

In [4]:
# Some subgoal
research_problem = '''Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

Evaluation
Goal
It is your job to predict the sales price for each house. For each Id in the test set, you must predict the value of the SalePrice variable. 

You want a train and validation MAE of lower than 11,000 and there should be a submission.csv containing predictions for test.csv ready to submit.

Metric
Submissions are evaluated on Mean-Absolute-Error (MAE) between the predicted value and the observed sales price.

Submission File Format
The file should contain a header and have the following format:

Id,SalePrice
1461,169000.1
1462,187724.1233
1463,175221
etc.'''

In [5]:
# Write the script manually

# Grabbing code that I know executes

python_code = '''
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

# Assuming 'train.csv' and 'test.csv' are in the working directory, read them into a pandas dataframe
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Remove rows with missing target, separate target from predictors
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = train_data.SalePrice
X = train_data.drop(['SalePrice'], axis=1)

# To keep things simple, we'll use only numerical predictors
X = X.select_dtypes(exclude=['object'])
test_data = test_data.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Simple imputation
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))
imputed_X_test = pd.DataFrame(my_imputer.transform(test_data))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns
imputed_X_test.columns = test_data.columns

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Fit model
model.fit(imputed_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(imputed_X_valid)
mae_valid = mean_absolute_error(y_valid, preds_valid)
print("Validation MAE:", mae_valid)

# If the model is satisfactory, we can train on all the data and predict the test set
if mae_valid < 15000:
    model.fit(pd.DataFrame(my_imputer.fit_transform(X)), y)
    test_preds = model.predict(imputed_X_test)
    output = pd.DataFrame({'Id': test_data.Id, 'SalePrice': test_preds})
    output.to_csv('submission.csv', index=False)
'''

write_args = {
    'file_name': f'train.py',
    'content': python_code,
    'update_files_action_result_history': False,
}
debug_agent.available_actions['writeFile'](**write_args)



--- LOGGING NEW ACTION ---
Step: 33
Calling function wrapped_write_file(args = (), kwargs = {'file_name': 'train.py', 'content': '\nimport pandas as pd\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.metrics import mean_absolute_error\nfrom sklearn.ensemble import RandomForestRegressor\n\n# Assuming \'train.csv\' and \'test.csv\' are in the working directory, read them into a pandas dataframe\ntrain_data = pd.read_csv(\'train.csv\')\ntest_data = pd.read_csv(\'test.csv\')\n\n# Remove rows with missing target, separate target from predictors\ntrain_data.dropna(axis=0, subset=[\'SalePrice\'], inplace=True)\ny = train_data.SalePrice\nX = train_data.drop([\'SalePrice\'], axis=1)\n\n# To keep things simple, we\'ll use only numerical predictors\nX = X.select_dtypes(exclude=[\'object\'])\ntest_data = test_data.select_dtypes(exclude=[\'object\'])\n\n# Break off validation set from training data\nX_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, te

'File train.py written successfully.'

In [6]:
# Execute the script
execute_args = {
    'script_name': 'train.py',
    'update_files_action_result_history': False,
}
result = debug_agent.env.execute_script(**execute_args)



--- LOGGING NEW ACTION ---
Step: 34
Calling function wrapped_execute_script(args = (), kwargs = {'script_name': 'train.py', 'work_dir': 'workspace\\home-data-for-ml-course_branch'})



Script output: Validation MAE: 18250.608013698627



--- LOGGING NEW ACTION ---
Step: 34
Calling function wrapped_complete_text_openai(args = (), kwargs = {'system_prompt': 'You are a helpful assistant. Your goal is to check if the code outputs the train and validation score, specifically not from log values, but normal values, and make sure that the code for calculating validation score is actually from a validation set. If so, then extract the train and validation score value from the result. If the code doesn\'t output the train and validation score or its not for normal values or the code doesn\'t actually calculate and print validation score from a validation set, then please write "inf" as the traing and validation score. Please also write down what type of validation score it is (ex. Mean Absolute Error (MAE), Root Mean Squared Error (RMSE), accuracy, etc.)\n            \n            Example:\n            ```json\n            {\n                "observations": "<string>",\n     

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>